In [ ]:
# Copyright 2025 Sony Corporation

In [ ]:
import subprocess
import os
import glob
import shutil
import sys

In [ ]:
workdir = os.getcwd()

In [ ]:
cwd = workdir

In [ ]:
BL_experiment_name = "BL_FlowSOM"
fsom_experiment_name = "Flowsom"
clust_res_dir = "BL_clustering_results"

In [ ]:
BL_iter_hang_ev = "evaluate_BL_hang.R"
FSOM_no_hang_seed_ev = "evaluate_FlowSOM_hang.R"
BL_meta_ev = "R/MetaClust.R"

In [ ]:
max_process = 30 #30 pallarell

## Set target samples

In [ ]:
Samplefcses = []
Samplefcses_pca = []

In [ ]:
Samplefcs = os.path.join(workdir,"benchmark_data_sets","Levine_32dim","data","Levine_32dim.fcs")
Samplefcses.append(Samplefcs)
Samplefcs = os.path.join(workdir,"benchmark_data_sets","Levine_13dim","data","Levine_13dim.fcs")
Samplefcses.append(Samplefcs)
Samplefcs = os.path.join(workdir,"benchmark_data_sets","Samusik_01","data","Samusik_01.fcs")
Samplefcses.append(Samplefcs)
Samplefcs = os.path.join(workdir,"benchmark_data_sets","Samusik_all","data","Samusik_all.fcs")
Samplefcses.append(Samplefcs)
Samplefcs = os.path.join(workdir,"benchmark_data_sets","FP34c","data","FP34c.fcs")
Samplefcses.append(Samplefcs)
Samplefcses_pca.append(Samplefcs)

In [ ]:
def do_evaluate_BL(BL_exp_path,eval_tool_path, Samplefcs,meta_list, it_list):
    os.chdir(eval_tool_path)
    loop = len(meta_list)*len(it_list)    
    sample_name = os.path.splitext(os.path.basename(Samplefcs))[0]
    proc_list = []
    total_num = 0
    for m in meta_list:
        print(m)
        for it in it_list:
            res_file = os.path.join(BL_exp_path, "fixit_%d_meta_%02dRes_%s_.metaclust"%(it,m,sample_name))
            fcs = Samplefcs
            f1_file  = os.path.join(BL_exp_path, "f1_meta_%02d_it_%02d_hang.hres"%(m,it))
            cmd = "RScript.exe %s %s %s %s"%(BL_iter_hang_ev, fcs, res_file, f1_file)
            proc = subprocess.Popen(cmd.split())
            proc_list.append(proc)
            f1_file  = os.path.join(BL_exp_path, "f1_%02d_hang.hres"%m)
            total_num = total_num +1
            if (total_num) % max_process == 0 or (total_num == loop):
                #wait for process
                for subproc in proc_list:
                    subproc.wait()
                proc_list = []

In [ ]:
def do_experiment_flowsom(fsom_exp_path, mode, sample_name,Samplefcs,meta_list, it_list,seed_num):
    os.makedirs(fsom_exp_path,exist_ok=True)
    loop = len(meta_list) * seed_num *len(it_list)
    init_mode = mode    
    proc_list = []
    total_num = 0
    for meta in meta_list:#meta
        print(meta)
        vseed_dir = os.path.join(fsom_exp_path, "%d"%meta) #exp_path/{meta}
        os.makedirs(vseed_dir,exist_ok=True)
        for it in it_list:#iteration
            it_dir = os.path.join( vseed_dir, "%d"%it)
            os.makedirs(it_dir,exist_ok=True)
            for i in range(1,seed_num+1):#seed
                cmd = "RScript.exe cytometry-clustering-comparison/run_methods/run_FlowSOM2_variable_iter_seed_meta_sample_jaccard.R %s %s %d %d %d %s %s"%(Samplefcs,it_dir, it, i, meta,sample_name,init_mode)
                proc = subprocess.Popen(cmd.split())
                proc_list.append(proc)
                total_num = total_num +1
                if (total_num) % max_process == 0 or (total_num == loop):
                    #wait for process
                    for subproc in proc_list:
                        subproc.wait()
                    proc_list = []    

In [ ]:
def do_evaluate_fsom(fsom_exp_path,eval_tool_path, Samplefcs,meta_list, it_list,seed_num):
    os.chdir(eval_tool_path)
    loop = len(meta_list)*len(it_list)*seed_num    
    sample_name = os.path.splitext(os.path.basename(Samplefcs))[0]
    proc_list = []
    total_num = 0
    for meta in meta_list:
        print(meta)
        fsom_eval_dir = os.path.join(fsom_exp_path, "%d"%meta) #exp_path/{meta}
        for it in it_list:
            it_dir = os.path.join(fsom_eval_dir, "%d"%it)
            for seed in range(1,seed_num + 1):
                cmd = "RScript.exe %s %s %s/FlowSOM_labels_seed_%d_iter_%d_meta_%d.txt %s/vseed_f1hang_%03d_iter_%d_meta_%d.resh"%(FSOM_no_hang_seed_ev,Samplefcs,it_dir,seed,it,meta,it_dir,seed,it,meta)
                proc = subprocess.Popen(cmd.split())
                proc_list.append(proc)
                total_num = total_num +1
                if (total_num) % max_process == 0 or (total_num == loop):
                    #wait for process
                    for subproc in proc_list:
                        subproc.wait()
                    proc_list = []
    print("done")

In [ ]:
def do_experiment_BL(BL_exp_path, sample_name, Samplefcs,meta_list,it_list):
    os.makedirs(BL_exp_path,exist_ok=True)
    out_dir = BL_exp_path
    clust_file = os.path.join(clust_res_dir,sample_name +"_clust.csv")
    codes_file  = os.path.join(clust_res_dir,sample_name +"_codes.csv")
    proc_list = []
    total_num = 0
    loop = len(meta_list)*len(it_list)
    for meta in meta_list:
        fname = "fixit_10_meta_%dRes_%s_.metaclust"%(meta,sample_name)
        out_fname = os.path.join(out_dir , fname)
        cmd = "RScript.exe %s %s %s %d %s"%(BL_meta_ev,clust_file,codes_file,meta, out_fname)
        proc = subprocess.Popen(cmd.split())
        proc_list.append(proc)
        total_num = total_num +1
        if (total_num) % max_process == 0 or (total_num == loop):
            #wait for process
            for subproc in proc_list:
                subproc.wait()
            proc_list = []

## BL-Flowsom experiment

In [ ]:
meta_list = range(10,61,1)
it_list = range(10,20,10)

In [ ]:
#please put Batch-learinig result in BL_clust_result folder

os.chdir(workdir)
max_process = 30 #para 

for Samplefcs in Samplefcses:
    sample_name = os.path.splitext(os.path.basename(Samplefcs))[0]
    BL_exp_path = os.path.join(workdir,BL_experiment_name,sample_name)
    # do BL-flowsom experiments
    do_experiment_BL(BL_exp_path,sample_name,Samplefcs,meta_list,it_list)
    os.chdir(workdir)
    eval_tool_path = os.path.join(cwd,"cytometry-clustering-comparison/evaluate_results")
    do_evaluate_BL(BL_exp_path,eval_tool_path,Samplefcs,meta_list,it_list)
    os.chdir(workdir)
os.chdir(workdir)

## flowsom experiment

In [ ]:
meta_list = range(10,61,1)
it_list = range(10,20,10)

In [ ]:
os.chdir(workdir)
seed_num=100
max_process = 30 

for Samplefcs in Samplefcses:
    sample_name = os.path.splitext(os.path.basename(Samplefcs))[0]
    mode = "RAND" #RAND/PCA
    fsom_exp_path = os.path.join(workdir,fsom_experiment_name,sample_name) 
    # do BL-flowsom experiments
    do_experiment_flowsom(fsom_exp_path,mode, sample_name,Samplefcs,meta_list,it_list,seed_num)
    os.chdir(workdir)
    eval_tool_path = os.path.join(workdir,"cytometry-clustering-comparison/evaluate_results")
    do_evaluate_fsom(fsom_exp_path,eval_tool_path,Samplefcs,meta_list,it_list,seed_num)
    os.chdir(workdir)
os.chdir(workdir)

In [ ]:
os.chdir(workdir)
#do flowsom with PCA
seed_num = 100
max_process = 20 

for Samplefcs in Samplefcses_pca:
    sample_name = os.path.splitext(os.path.basename(Samplefcs))[0] + "-pca" #add sample name to pca 
    mode = "PCA" #RAND/PCA
    fsom_exp_path = os.path.join(workdir,fsom_experiment_name,sample_name) 
    # do BL-flowsom experiments
    do_experiment_flowsom(fsom_exp_path,mode, sample_name,Samplefcs,meta_list,it_list,seed_num)
    os.chdir(workdir)
    eval_tool_path = os.path.join(workdir,"cytometry-clustering-comparison/evaluate_results")
    do_evaluate_fsom(fsom_exp_path,eval_tool_path,Samplefcs,meta_list,it_list,seed_num)
os.chdir(workdir)